# GPU computing in Julia

## Overview of the landscape

- [JuliaGPU](https://juliagpu.org/): Mother organisation
- [CUDA](https://github.com/JuliaGPU/CUDA.jl): NVIDIA GPUs, best supported, focus of this talk
- [oneAPI](https://github.com/JuliaGPU/oneAPI.jl): Support for Intel GPUs with oneAPI (slightly less functionality and performance)
- [AMDGPU](https://github.com/JuliaGPU/AMDGPU.jl): AMD GPUs running on the ROCm stack (experimental)

GPU support is heavy work in progress, but user experience with NVIDIA GPUs is already smooth and using them can be highly recommended. With other platforms I'd be careful as of now. That's why we will focus on CUDA.jl here.

## Installation

Install CUDA package:

In [ ]:
import Pkg
Pkg.add("CUDA")

Important difference to most other Julia packages: This does *not yet* install the full thing. The reason is that not every computer has a GPU, but packages should still do sth. sensible.

We can check wether we have a GPU:

In [ ]:
using CUDA
if CUDA.functional()
    println("Congrats! You have CUDA-enabled GPU!")
else
    println("Sorry no GPU support detected")
end

(I will assume from here on that we have a GPU ... most of what I show will crash if you don't)

Let's see some info about our device:

In [ ]:
CUDA.versioninfo()

## High-level tour

The CUDA.jl package provides equivalent GPU functions and datastructures for a large portion of the standard library.

In [ ]:
A = CUDA.randn(3, 3)  # Random matrix

In [ ]:
b = CUDA.randn(3)   # Random vector
A * b   # Matrix-Vector product

In [ ]:
sum(A, dims=2)   # Sum over second dimension

In [ ]:
A[:, 2]   # Second column

In [ ]:
A \ b   # Solve dense linear system

Notice: The default data type on GPUs is **Float32**!

GPU discouraged features (e.g. scalar indexing) are allowed by default:

In [ ]:
A[1,1] = 2.3

In [ ]:
CUDA.allowscalar(false)  # Disable scalar indexing

In [ ]:
A[1,1] = 2.3

In [ ]:
A[1, :] = A[2, :]  # Slice indexing still allowed!

### Example: Power iteration

As demonstrated above, there is barely any difference between GPU and CPU Julia code ... and that's the great power! Let's see this in practice:

In [ ]:
function power_iteration(A, x; tol=1e-4, maxiter=100, display_progress=true)
    x  = x / norm(x)
    Ax = similar(x)  # Allocate scratch memory
    λ  = zero(eltype(Ax))
    for i = 1:maxiter
        mul!(Ax, A, x)
        λ = x'Ax
        display_progress && println("iter $i $λ")
        Ax ./= norm(Ax)
        sqrt(abs(1 - Ax'x)) < tol && return λ, Ax
        x .= Ax
    end
    λ, x
end

Use it with CPUs as usual:

In [ ]:
A = randn(Float32, 100, 100)
A = A + A' + 10I
x = rand(Float32, 100)

λ, _ = power_iteration(A, x, display_progress=false)
λ

Use it on the GPU:

In [ ]:
λ, _ = power_iteration(cu(A), cu(x), display_progress=false)
λ

### Example: Poisson's equation on the GPU

We wish to solve $ \Delta x = ρ $ on $[0, 1]$ where
$ ρ(r) = δ(x) -2δ(x - 0.5)$. We discretise using finite differences using $100$ grid points:

In [ ]:
using SparseArrays
N = 101

# Prepare input on the CPU
Δ = spdiagm(0 => 2ones(Float32, N), 1=>-ones(Float32, N-1), -1 => -ones(Float32, N-1))
ρ = zeros(N)
ρ[1] = 1
ρ[Int((N-1)/2)] = -2

# Send to device:
Δ_d = cu(Δ)
ρ_d = cu(ρ)

Δ_d   # Note: Structure is kept!

In [ ]:
using IterativeSolvers

# Solve using a CG (on the device!)
x = cg(Δ_d, ρ_d, verbose=true)

## Low-level tour

### Unsupported functions

Many CUDA functions are directly supported from the high-level Julia interfaces. Unfortunately some are not. Some examples.

In [ ]:
# LAPACK-like call from CUSOLVER

A = CUDA.randn(4, 4)
A = A + A'
eigen(A)  # Oh that would be so nice ...

In [ ]:
# But this works ...
λ, v = CUDA.CUSOLVER.syevd!('V', 'U', A)   # Like LAPACKs "syevd"
@show λ

### Custom kernels

This is where the real power starts ...

In [ ]:
function gpuadd_sequential!(y, x)
    for i = 1:length(y)
        @inbounds y[i] += x[i]
    end
end

In [ ]:
y = 2CUDA.ones(1000)
x =  CUDA.ones(1000)
@cuda gpuadd_sequential!(y, x)  # Call the kernel
all(Array(y) .== 3.0f0)

How long did that take?

In [ ]:
@btime CUDA.@sync @cuda gpuadd_sequential!($y, $x);

Parallelise ...

In [ ]:
function gpuadd!(y, x)
    index  = threadIdx().x
    stride = blockDim().x    
    for i = index:stride:length(y)
        @inbounds y[i] += x[i]
    end
end

y = 2CUDA.ones(1000)
x =  CUDA.ones(1000)
@cuda threads=256 gpuadd!(y, x)  # Call the kernel
all(Array(y) .== 3.0f0)

In [ ]:
@btime CUDA.@sync @cuda threads=256 gpuadd!($y, $x);

Kernels for both GPU and CPU: [KernelAbstractions.jl](https://github.com/JuliaGPU/KernelAbstractions.jl)

### See what's going on

- `nvidia-smi`
- NSight Systems `nsys` and `CUDA.@profile`

## Overview: What is supported

- General factorisations (Cholesky, QR, ...)
- Dense linear solves
- Sparse arrays
- CUDA FFTs
- Most BLAS, LAPACK
- Plenty of nice packages (IterativeSolvers)
- [Profiling](https://juliagpu.github.io/CUDA.jl/stable/development/profiling/)
- ...

## Closing example: LOBPCG

So how would one go about implement something on the GPU. We will take a look at [lobpcg.jl](lobpcg.jl), which is a (crappy) CPU version of a sophisticated iterative diagonalisation algorithm. To implement it on the GPU these are the steps to follow:

1. Implement it on CPU (done)
2. Profiling and performance optimisation on CPU (skipped ... but really should be done first!)
3. Get it to work on the GPU
4. Turn off scalar indexing, fix all issues
5. Profile on GPU
6. Implement custom kernels if needed

Ok, so let's go with step 3 ...